In [1]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [3]:
from src.api.common.dependency_container import DependencyContainer
DC = DependencyContainer(); DC.initialize()

In [10]:
pregunta = 'Hola,mi nombre es Gloria,o?'
request = {'Pregunta': pregunta, 'FAQs': faqs + faqs_unir}  
response = DependencyContainer.get_faqs_workflow().execute(request)
faq = max(response['FAQs'], key=lambda x: float(x['Score']))
if float(faq['Score']) >= 7: content = f"<h2>{faq['FAQ']}</h2>{faq['Contenido']}" 
if 4 < float(faq['Score']) < 7: 
    content = f"""
                <h2>{faq['FAQ']}</h2>{faq['Contenido']}
                <h2>Si esta FAQ no se ajusta a tu pregunta, puedes dirigirte al siguiente número de Whatsapp: <a href="https://web.whatsapp.com/send?phone=34689909323">Contacta con un asesor.</a></h2>
    """
else: content = f'<h2>No encontramos ninguna FAQ relacionada a la pregunta. Dirígete al siguiente número de Whatsapp: <a href="https://web.whatsapp.com/send?phone=34689909323">contacta con un asesor.</a></h2>'
response['FAQs']

[{'FAQ': '0. Olvidé mi contraseña',
  'Contenido': '<div><p>El sistema más sencillo de recuperarla (es preciso configurarla de nuevo) es el que se muestra en la imagen:</p><p class="centrar"><img alt="Contraseña perdida" class="responsive" src="https://becat.online/FAQ/img/perdida-pwd-1.jpg"/></p><p>Recibirás un correo como el siguiente, que permitirá reconfigurar la contraseña:</p><p class="centrar"><img alt="Contraseña perdida 2" class="responsive" src="https://becat.online/FAQ/img/perdida-pwd-2.jpg"/></p></div>',
  'Razonamiento': 'No hay relación entre la pregunta y esta FAQ.',
  'Score': '0'},
 {'FAQ': '1. No puedo acceder con mis datos de usuario',
  'Contenido': '<div><p>Asegúrate en primer lugar que introduces bien los datos:</p><ul>\n<li><strong>Nombre de usuario:</strong> tu correo electrónico. En el ejemplo es "lidianarpaz@yahoo.com": sería un error poner un espacio más "lidianarpaz@yahoo.com " al final, o en medio o empezar con mayúscula. </li>\n<li><strong>Contraseña:</str